In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [3]:
farmacias = pd.read_csv('datasets/mediPy/farmacia_popular_estabelecimentonone.csv')

In [4]:
farmacias.head()

,lat,long,nu_ddd_farmacia,nu_telefone_farmacia,nu_cep_farmacia,no_bairro_farmacia,ds_endereco_farmacia,no_farmacia,no_cidade,uf
0,-32.236851,-53.088458,54,33131900,96330000,CENTRO,"AVENIDA VISCONDE DE MAUA, 383",COMERCIO DE MEDICAMENTOS BRAIR LTDA,ARROIO GRANDE,RS
1,-31.753500,-52.312792,54,33350100,96085470,AREAL,AVENIDA DOMINGOS DE ALMEIDA 825 LOJA 01,COMERCIO DE MEDICAMENTOS BRAIR LTDA,PELOTAS,RS
2,-23.084332,-47.215462,19,38345604,13330060,CENTRO,"RUA VINTE E QUATRO DE MAIO, 1036",DROGAN DROGARIAS LTDA,INDAIATUBA,SP
3,-27.081033,-53.027768,49,33254232,89885000,CENTRO,"AVENIDA SANTA CATARINA, Nº 101 - A",DROGARIA PATRICIA LTDA,SAO CARLOS,SC
4,-12.993305,-38.457390,11,32747400,41810315,PITUBA,RUA ARISTIDES FRAGA LIMA 51,DROGARIA SAO PAULO S.A.,SALVADOR,BA


In [5]:
del farmacias['nu_ddd_farmacia']
del farmacias['nu_telefone_farmacia']
del farmacias['no_bairro_farmacia']
del farmacias['ds_endereco_farmacia']
del farmacias['no_cidade']

In [6]:
farmacias.head()

,lat,long,nu_cep_farmacia,no_farmacia,uf
0,-32.236851,-53.088458,96330000,COMERCIO DE MEDICAMENTOS BRAIR LTDA,RS
1,-31.753500,-52.312792,96085470,COMERCIO DE MEDICAMENTOS BRAIR LTDA,RS
2,-23.084332,-47.215462,13330060,DROGAN DROGARIAS LTDA,SP
3,-27.081033,-53.027768,89885000,DROGARIA PATRICIA LTDA,SC
4,-12.993305,-38.457390,41810315,DROGARIA SAO PAULO S.A.,BA


In [7]:
farmacias.sort_values('uf',inplace = True)

In [8]:
farmacias.head()

,lat,long,nu_cep_farmacia,no_farmacia,uf
5139,-9.975700,-67.801836,69900064,L B OLIVEIRA LTDA - EPP,AC
2549,-9.965569,-67.832388,69912600,FARMACIA MARANATA LTDA,AC
6777,-9.974607,-67.872819,69905596,J SOL LTDA - EPP,AC
26618,-9.975700,-67.801836,69900072,J CRUZ LTDA - EPP,AC
27266,-3.729726,-38.498132,69932000,G. N. DE CARVALHO IMP E EXP - ME,AC


In [9]:
def quantidade_uf():
    quantidades_ufs = {}
    for index in range(0, len(farmacias)):
            uf = farmacias.loc[index]['uf']
            if len(str(uf))==2:
                if uf not in quantidades_ufs:
                    quantidades_ufs[uf] = 1
                else:
                    quantidades_ufs[uf] = quantidades_ufs[uf] + 1
    return quantidades_ufs

In [10]:
ufs = quantidade_uf()

In [11]:
ufs

{'AC': 20,
 'AL': 145,
 'AM': 46,
 'AP': 6,
 'BA': 1078,
 'CE': 514,
 'DF': 410,
 'ES': 922,
 'GO': 1636,
 'MA': 237,
 'MG': 5155,
 'MS': 431,
 'MT': 552,
 'PA': 355,
 'PB': 484,
 'PE': 488,
 'PI': 199,
 'PR': 2723,
 'RJ': 1897,
 'RN': 486,
 'RO': 249,
 'RR': 48,
 'RS': 3151,
 'SC': 1867,
 'SE': 129,
 'SP': 4653,
 'TO': 145}

In [12]:
farmacias = farmacias[~farmacias.no_farmacia.isnull()]

In [13]:
farmacias

,lat,long,nu_cep_farmacia,no_farmacia,uf
5139,-9.975700,-67.801836,69900064,L B OLIVEIRA LTDA - EPP,AC
2549,-9.965569,-67.832388,69912600,FARMACIA MARANATA LTDA,AC
6777,-9.974607,-67.872819,69905596,J SOL LTDA - EPP,AC
26618,-9.975700,-67.801836,69900072,J CRUZ LTDA - EPP,AC
27266,-3.729726,-38.498132,69932000,G. N. DE CARVALHO IMP E EXP - ME,AC
21626,-9.965569,-67.832388,69912600,EMPREENDIMENTOS PAGUE MENOS S/A,AC
1671,-9.973320,-67.824191,69906383,J H RODRIGUES - EPP,AC
8802,-8.026177,-72.804280,69980000,OTILIA B. DE OLIVEIRA - EPP,AC
17396,-9.929761,-67.839094,69918360,SANTOS & SILVA LTDA - ME,AC
19498,-9.929761,-67.839094,69918172,CRUZ & OLIVEIRA LTDA - EPP,AC


In [14]:
farmacias = farmacias[~farmacias.uf.isnull()]

In [15]:
farmacias

,lat,long,nu_cep_farmacia,no_farmacia,uf
5139,-9.975700,-67.801836,69900064,L B OLIVEIRA LTDA - EPP,AC
2549,-9.965569,-67.832388,69912600,FARMACIA MARANATA LTDA,AC
6777,-9.974607,-67.872819,69905596,J SOL LTDA - EPP,AC
26618,-9.975700,-67.801836,69900072,J CRUZ LTDA - EPP,AC
27266,-3.729726,-38.498132,69932000,G. N. DE CARVALHO IMP E EXP - ME,AC
21626,-9.965569,-67.832388,69912600,EMPREENDIMENTOS PAGUE MENOS S/A,AC
1671,-9.973320,-67.824191,69906383,J H RODRIGUES - EPP,AC
8802,-8.026177,-72.804280,69980000,OTILIA B. DE OLIVEIRA - EPP,AC
17396,-9.929761,-67.839094,69918360,SANTOS & SILVA LTDA - ME,AC
19498,-9.929761,-67.839094,69918172,CRUZ & OLIVEIRA LTDA - EPP,AC
